##  Importing the required libraries

In [16]:
import os
import datetime
import pydot
import tensorflow as tf
import h5py
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np
from contextlib import redirect_stdout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.utils import to_categorical

# Load the TensorBoard notebook extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


## Building the VGG16 model

### 1) With functional api.

In [17]:
def vgg16(input_shape = None, classes = None):
    if input_shape and classes is not None:
        input_data = tf.keras.layers.Input(shape=input_shape)
        # First Convolutional block
        conv1_1 = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),
                                         padding="same", activation="relu")(input_data)
        conv1_2 = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),
                                         padding="same", activation="relu")(conv1_1)
        maxpool_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(conv1_2)
        
        # Second Convolution Block
        conv2_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),
                                         padding="same", activation="relu")(maxpool_1)
        conv2_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv2_1)
        maxpool_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(conv2_2)
        
        # Third Convolution Block
        conv3_1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),
                                         padding="same", activation="relu")(maxpool_2)
        conv3_2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv3_1)
        conv3_3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv3_2)
        maxpool_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(conv3_3)
        
        # Fourth Convolutional Block
        conv4_1 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")(maxpool_3)
        conv4_2 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv4_1)
        conv4_3 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv4_2)
        maxpool_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(conv4_3)
        
        # Fifth Convolutional Block
        conv5_1 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")(maxpool_4)
        conv5_2 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv5_1)
        conv5_3 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")(conv5_2)
        maxpool_5 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))(conv5_3)
        
        # Flatten
        flatten = tf.keras.layers.Flatten()(maxpool_5)
        fc1 = tf.keras.layers.Dense(4096, activation='relu')(flatten)
        fc2 = tf.keras.layers.Dense(4096, activation='relu')(fc1)
        
        output_data = tf.keras.layers.Dense(classes, activation='softmax')(fc2)
        
        model = tf.keras.models.Model(inputs = input_data, outputs = output_data)
        
        return model
        

#### a) Model Summary

In [19]:
 def summary(output=None, target=None, model = None):
        """ Show / Save model structure (summary) """
        if model is not None:
            model.summary()
            if target is not None:
                os.makedirs(output, exist_ok=True)
                with open(os.path.join(output, target), "w") as f:
                    with redirect_stdout(f):
                        model.summary()

In [20]:
model = vgg16(input_shape=(224, 224, 3), classes = 1000)
summary(output= './summary', target='summary', model = model)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 56, 56, 128)       0   

#### b) Model visual_summary

In [21]:
def model_plot(model = None, file_name=None):
    if model and file_name is not None:
        print('Model plotting...')
        
        plot_model(model, to_file = file_name, show_shapes=True,
                   show_layer_names=True,
                   rankdir='TB', expand_nested=True, dpi=96)
        print('Model plotted..')

In [22]:
model_plot(model = model, file_name='vgg16_model.png')

Model plotting...
Model plotted..


## data preprocessing

In [23]:
def preprocessing():
    train_dataset = h5py.File('dataset/train_signs.h5', "r")
    train_set_x_orig = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_set_y_orig = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('dataset/test_signs.h5', "r")
    test_set_x_orig = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_set_y_orig = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes


    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes


def load_data():
    train_x_orig, train_y, test_x_orig, test_y, classes = preprocessing()
    
    # Standardize data to have feature values between 0 and 1.
    train_x = train_x_orig/255.
    test_x = test_x_orig/255.
    
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(train_x[i], cmap=plt.cm.binary)
        plt.xlabel(train_y[i])
    plt.show()

    return train_x, train_y, test_x, test_y


    

In [24]:
train_x, train_y, test_x, test_y = load_data()
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_x, test_y)).batch(32)

OSError: Unable to open file (unable to open file: name = 'dataset/train_signs.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
print(train_x.shape)
print(train_y.shape)

### 2) Custom_Subclassing

#### a) Custom dense layer

In [26]:
class Dense(tf.keras.layers.Layer):

  def __init__(self, units=32, initializer=None):
    super(Dense, self).__init__()
    self.units = units
    self.initializer = initializer

  def build(self, input_shape):
    w_init = self.initializer
    self.w = tf.Variable(initial_value=w_init(shape=(input_shape[-1], self.units),
                         dtype='float32'),trainable=True)
    b_init = self.initializer
    self.b = tf.Variable(initial_value=b_init(shape=(self.units,),
                         dtype='float32'),trainable=True)
   
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

  def get_config(self):
    config = super(Dense, self).get_config()
    config.update({'units': self.units})
    return config

In [27]:
dense = Dense(units = 80, initializer = tf.initializers.GlorotUniform())
x = tf.ones((2, 2))
y = dense(x)
print(y.shape)

(2, 80)


In [28]:
print('weights:', len(dense.weights))
print('trainable weights:', len(dense.trainable_weights))  # w and b

weights: 2
trainable weights: 2


In [29]:
config = dense.get_config()
config

{'name': 'dense_7', 'trainable': True, 'dtype': 'float32', 'units': 80}

#### b) Custom dropout layer

In [30]:
class Dropout(tf.keras.layers.Layer):
    def __init__(self, rate, **kwargs):
        super(Dropout, self).__init__(**kwargs)
        self.rate = rate

    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

In [31]:
dropout = Dropout(0.5)
y = dropout(y, training=True)
print(y.shape)

(2, 80)


## Model

In [32]:
class Model(tf.keras.Model):
    def __init__(self, inputShape, classes):
        super(Model, self).__init__()
        self.inputShape = inputShape
        self.classes = classes
        
        # First Convolutional block
        self.conv1_1 = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),
                                         input_shape=inputShape,padding="same",
                                         activation="relu")
        self.conv1_2 = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.maxpool_1 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))
        
        # Second Convolution Block
        self.conv2_1 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.conv2_2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.maxpool_2 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))
        
        # Third Convolution Block
        self.conv3_1 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.conv3_2 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.conv3_3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.maxpool_3 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))
        
        # Fourth Convolutional Block
        self.conv4_1 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.conv4_2 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.conv4_3 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")
        
        self.maxpool_4 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))
        
        # Fifth Convolutional Block
        self.conv5_1 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")
        self.conv5_2 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")
        self.conv5_3 = tf.keras.layers.Conv2D(filters=512, kernel_size=(3,3),
                                         padding="same", activation="relu")
        self.maxpool_5 = tf.keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2))
        
        # Flatten
        self.flatten = tf.keras.layers.Flatten()
        
        self.fc1 = Dense(units=4096, initializer=tf.initializers.GlorotUniform())
        self.act1 = tf.keras.layers.ReLU()
        self.drop1 = Dropout(0.5)
        
        self.fc2 = Dense(units=4096, initializer=tf.initializers.GlorotUniform())
        self.act2 = tf.keras.layers.ReLU()
        self.drop2 = Dropout(0.5)
        
        self.output_data = Dense(units=self.classes, initializer=tf.initializers.GlorotUniform())
        self.act3 = tf.keras.layers.Softmax()
        
    def call(self, x):
        # First
        x = self.conv1_1(x)
        x = self.conv1_2(x)
        x = self.maxpool_1(x)
        
        # Second Convolution Block
        x = self.conv2_1(x)
        x = self.conv2_2(x)
        x = self.maxpool_2(x)
        
        # Third Convolution Block
        x = self.conv3_1(x)
        x = self.conv3_2(x)
        x = self.conv3_3(x)
        x = self.maxpool_3(x)
        
        # Fourth Convolutional Block
        x = self.conv4_1(x)
        x = self.conv4_2(x)
        x = self.conv4_3(x)
        x = self.maxpool_4(x)
        
        # Fifth Convolutional Block
        x = self.conv5_1(x)
        x = self.conv5_2(x)
        x = self.conv5_3(x)
        x = self.maxpool_5(x)
        
        # Flatten
        x = self.flatten(x)
        
        x = self.fc1(x)
        x = self.act1(x)
        x = self.drop1(x, training=True)
        
        x = self.fc2(x)
        x = self.act2(x)
        x = self.drop2(x, training=True)
        
        x = self.output_data(x)
        x = self.act3(x)

        return x


In [33]:
input_shape = (224, 224, 3)
classes = 1000
m = Model(input_shape, classes)

#### Custom_Model summary

In [34]:
input_shape = (1, 224, 224, 3)
m.build(input_shape)

In [35]:
summary(output= './summary', target='custom_model_summary', model = m)

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           multiple                  1792      
_________________________________________________________________
conv2d_27 (Conv2D)           multiple                  36928     
_________________________________________________________________
max_pooling2d_10 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_28 (Conv2D)           multiple                  73856     
_________________________________________________________________
conv2d_29 (Conv2D)           multiple                  147584    
_________________________________________________________________
max_pooling2d_11 (MaxPooling multiple                  0         
_________________________________________________________________
conv2d_30 (Conv2D)           multiple                  2951

In [36]:
class NN:
    def __init__(self):
        self.loss = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.Adam()
        self.train_loss = tf.keras.metrics.Mean(name= 'train_loss')
        self.train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
        self.test_loss = tf.keras.metrics.Mean(name='test_loss')
        self.test_accuracy = tf.keras.metrics.CategoricalAccuracy(name='test_accuracy')

        self.model = Model((224,224, 3),1000)

    @tf.function
    def train_step(self, inputs, outputs):
        with tf.GradientTape() as tape:
            predictions = self.model(inputs, training=True)
            loss = self.loss(outputs, predictions)
        gradients = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        self.train_loss(loss)
        self.train_accuracy(outputs, predictions)


        
    @tf.function
    def test_step(self, inputs, outputs):
        predictions = self.model(inputs)
        t_loss = self.loss(outputs, predictions)
        self.test_loss(t_loss)
        self.test_accuracy(outputs, predictions)


    
    def train(self, epochs=1, train_ds = None, test_ds = None):
        for epoch in range(epochs):
            for inputs, outputs in train_ds:
                self.train_step(inputs, outputs)

            for test_inputs, test_outputs in test_ds:
                self.test_step(test_inputs, test_outputs)



            template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy:{}'
            print(template.format(epoch+1,self.train_loss.result(),self.train_accuracy.result()*100,self.test_loss.result(),self.test_accuracy.result()*100))

        self.train_loss.reset_states()
        self.train_accuracy.reset_states()
        self.test_loss.reset_states()
        self.test_accuracy.reset_states()


    def save(self, name='', save_format='tf'):
        self.model.save(name, save_format=save_format)
        return "Your model saved"
        

    def predict(self, input):
        prediction = self.model.predict(input)
        
        return prediction






        




In [37]:
nn = NN()